# Ray Tracing: Is our analytical approach going to work?

# Fifth question: Computation of the time delays and the amplitude ratios for a specific array and a specific location of the tremor source.

Name of the array.

In [1]:
name = 'BS'

Location of the tremor source.

In [2]:
lat = 47.95728
lon = - 122.92866
depth = 41487.7

Load Python modules.

In [3]:
import numpy
from math import sqrt, pi, cos, sin, tan

Load functions from my own ray-tracing related modules.

In [4]:
from computeAmplitude import computeAmplitude3SH, computeAmplitude5SH, computeAmplitude3PSV, computeAmplitude5PSV
from computeAngle import computeAngle3, computeAngle5
from computeTravelTime import computeTravelTime3, computeTravelTime5
from misc import computeInitAmp, latLon2xy

Get P- and S-wave velocities in the continental crust.

In [5]:
from data import VpCC, VsCC

Get strike and dip of the subducted oceanic plate.

In [6]:
from data import phi, delta

Get location of array and convert the distance into meters.

In [7]:
from data import lat_a, lon_a, names_a
mylat = lat_a[names_a == name]
mylon = lon_a[names_a == name]
(x, y) = latLon2xy(mylat, mylon, lat, lon)

Displacement of the tremor source.

In [8]:
x0 = - cos(phi * pi / 180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])
y0 = sin(phi * pi / 180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])
d0 = depth - tan(delta * pi /180.0) * numpy.array([0.0, 500.0, 1800.0, 3300.0])

Types of wave.

In [9]:
wave = ('P', 'S')

Initializations.

In [10]:
tP = numpy.zeros((numpy.shape(x0)[0]))
tS = numpy.zeros((numpy.shape(x0)[0]))
t3SH = numpy.zeros((numpy.shape(x0)[0]))
t5SH = numpy.zeros((numpy.shape(x0)[0]))
t3PSV = numpy.zeros((numpy.shape(x0)[0], 8))
t5PSV = numpy.zeros((numpy.shape(x0)[0], 32))
A3PSV = numpy.zeros((8))
A5PSV = numpy.zeros((32))

Open output file.

In [11]:
output = open('RayTracing.txt', 'w')

Loop on source position.

In [12]:
for i in range(0, numpy.shape(x0)[0]):
    # Direct wave
    tP[i] = sqrt((x - x0[i]) ** 2.0 + (y - y0[i]) ** 2.0 + d0[i] ** 2.0) / VpCC
    tS[i] = sqrt((x - x0[i]) ** 2.0 + (y - y0[i]) ** 2.0 + d0[i] ** 2.0) / VsCC
    if i == 0:
        AD = computeInitAmp(x0[i], y0[i], x, y, d0[i], 'D')
        AP = AD[0]
        ASV = AD[1]
        ASH = AD[2]
    # Reflected SH-wave on mid-slab discontinuity
    angle = computeAngle3(x0[i], y0[i], d0[i], x, y, 'S', 'S', 'S')
    t3SH[i] = computeTravelTime3(angle, x0[i], y0[i], d0[i], x, y, 'S', 'S', 'S')
    if i == 0:
        A3SH = computeAmplitude3SH(angle, x0[i], y0[i], x, y)
        AR = computeInitAmp(x0[i], y0[i], x, y, d0[i], 'R', angle)
        A3SH = A3SH * AR[2]
    # Reflected SH-wave on Moho
    angle = computeAngle5(x0[i], y0[i], d0[i], x, y, 'S', 'S', 'S', 'S', 'S')
    t5SH[i] = computeTravelTime5(angle, x0[i], y0[i], d0[i], x, y, 'S', 'S', 'S', 'S', 'S')
    if i == 0:
        A5SH = computeAmplitude5SH(angle, x0[i], y0[i], x, y)
        AR = computeInitAmp(x0[i], y0[i], x, y, d0[i], 'R', angle)
        A5SH = A5SH * AR[2]
    # Reflected wave on mid-slab discontinuity
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Upgoing wave in upper oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in continental crust
            for k3 in range(0, 2):
                k = k1 * 1 + k2 * 2 + k3 * 4
                angle = computeAngle3(x0[i], y0[i], d0[i], x, y, wave[k1], wave[k2], wave[k3])
                t3PSV[i, k] = computeTravelTime3(angle, x0[i], y0[i], d0[i], x, y, wave[k1], wave[k2], wave[k3])
                if i == 0:
                    A3PSV[k] = computeAmplitude3PSV(angle, x0[i], y0[i], x, y, wave[k1], wave[k2], wave[k3])
                    AR = computeInitAmp(x0[i], y0[i], x, y, d0[i], 'R', angle)
                    if (k1 == 0):
                        A3PSV[k] = A3PSV[k] * AR[0]
                    else:
                        A3PSV[k] = A3PSV[k] * AR[1]
    # Reflected wave on Moho
    # Downgoing wave in upper oceanic crust
    for k1 in range(0, 2):
        # Downgoing wave in lower oceanic crust
        for k2 in range(0, 2):
            # Upgoing wave in lower oceanic crust
            for k3 in range(0, 2):
                # Upgoing wave in upper oceanic crust
                for k4 in range(0, 2):
                    # Upgoing wave in continental crust
                    for k5 in range(0, 2):
                        k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                        angle = computeAngle5(x0[i], y0[i], d0[i], x, y, \
                                wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                        t5PSV[i, k] = computeTravelTime5(angle, x0[i], y0[i], d0[i], x, y, \
                                      wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                        if i == 0:
                            A5PSV[k] = computeAmplitude5PSV(angle, x0[i], y0[i], x, y, \
                                       wave[k1], wave[k2], wave[k3], wave[k4], wave[k5])
                            AR = computeInitAmp(x0[i], y0[i], x, y, d0[i], 'R', angle)
                            if (k1 == 0):
                                A5PSV[k] = A5PSV[k] * AR[0]
                            else:
                                A5PSV[k] = A5PSV[k] * AR[1]

output.write('Time lag between direct waves\n')
output.write('Amplitude ratio of direct SV-wave / direct P-wave: {}\n'.format(ASV / AP))
output.write('Time lag between direct SV-wave and direct P-wave: {}\n'.format(tS[0] - tP[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (tS[1] - tP[1]) - (tS[0] - tP[0]), \
    (tS[2] - tP[2]) - (tS[0] - tP[0]), \
    (tS[3] - tP[3]) - (tS[0] - tP[0])))
output.write('\n')
output.write('Amplitude ratio of direct SH-wave / direct P-wave: {}\n'.format(ASH / AP))
output.write('Time lag between direct SH-wave and direct P-wave: {}\n'.format(tS[0] - tP[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (tS[1] - tP[1]) - (tS[0] - tP[0]), \
    (tS[2] - tP[2]) - (tS[0] - tP[0]), \
    (tS[3] - tP[3]) - (tS[0] - tP[0])))
output.write('\n')

output.write('Time lag with direct P-wave\n')
output.write('Amplitude ratio of SH-wave (mid-slab) / direct P-wave: {}\n'.format(A3SH / AP))
output.write('Time lag between SH-wave (mid-slab) and direct P-wave: {}\n'.format(t3SH[0] - tP[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (t3SH[1] - tP[1]) - (t3SH[0] - tP[0]), \
    (t3SH[2] - tP[2]) - (t3SH[0] - tP[0]), \
    (t3SH[3] - tP[3]) - (t3SH[0] - tP[0])))
output.write('\n')
output.write('Amplitude ratio of SH-wave (Moho) / direct P-wave: {}\n'.format(A5SH / AP))
output.write('Time lag between SH-wave (Moho) and direct P-wave: {}\n'.format(t5SH[0] - tP[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (t5SH[1] - tP[1]) - (t5SH[0] - tP[0]), \
    (t5SH[2] - tP[2]) - (t5SH[0] - tP[0]), \
    (t5SH[3] - tP[3]) - (t5SH[0] - tP[0])))
output.write('\n')

# Reflected wave on mid-slab discontinuity
# Downgoing wave in upper oceanic crust
for k1 in range(0, 2):
    # Upgoing wave in upper oceanic crust
    for k2 in range(0, 2):
        # Upgoing wave in continental crust
        for k3 in range(0, 2):
            k = k1 * 1 + k2 * 2 + k3 * 4
            output.write('Amplitude ratio of ray {}{}{} / direct P-wave: {}\n'.format(wave[k1], wave[k2], wave[k3], \
                A3PSV[k] / AP))
            output.write('Time lag between ray {}{}{} and direct P-wave: {}\n'.format(wave[k1], wave[k2], wave[k3], \
                t3PSV[0, k] - tP[0]))
            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                (t3PSV[1, k] - tP[1]) - (t3PSV[0, k] - tP[0]), \
                (t3PSV[2, k] - tP[2]) - (t3PSV[0, k] - tP[0]), \
                (t3PSV[3, k] - tP[3]) - (t3PSV[0, k] - tP[0])))
            output.write('\n')

# Reflected wave on Moho
# Downgoing wave in upper oceanic crust
for k1 in range(0, 2):
    # Downgoing wave in lower oceanic crust
    for k2 in range(0, 2):
        # Upgoing wave in lower oceanic crust
        for k3 in range(0, 2):
            # Upgoing wave in upper oceanic crust
            for k4 in range(0, 2):
                # Upgoing wave in continental crust
                for k5 in range(0, 2):
                    k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                    output.write('Amplitude ratio of ray {}{}{}{}{} / direct P-wave: {}\n'.format(wave[k1], wave[k2], \
                        wave[k3], wave[k4], wave[k5], A5PSV[k] / AP))
                    output.write('Time lag between ray {}{}{}{}{} and direct P-wave: {}\n'.format(wave[k1], wave[k2], \
                        wave[k3], wave[k4], wave[k5], t5PSV[0, k] - tP[0]))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t5PSV[1, k] - tP[1]) - (t5PSV[0, k] - tP[0]), \
                        (t5PSV[2, k] - tP[2]) - (t5PSV[0, k] - tP[0]), \
                        (t5PSV[3, k] - tP[3]) - (t5PSV[0, k] - tP[0])))
                    output.write('\n')

output.write('Time lag with direct S-wave\n')
output.write('Amplitude ratio of SH-wave (mid-slab) / direct S-wave: {}\n'.format(A3SH / ASH))
output.write('Time lag between SH-wave (mid-slab) and direct S-wave: {}\n'.format(t3SH[0] - tS[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (t3SH[1] - tS[1]) - (t3SH[0] - tS[0]), \
    (t3SH[2] - tS[2]) - (t3SH[0] - tS[0]), \
    (t3SH[3] - tS[3]) - (t3SH[0] - tS[0])))
output.write('\n')
output.write('Amplitude ratio of SH-wave (Moho) / direct S-wave: {}\n'.format(A5SH / ASH))
output.write('Time lag between SH-wave (Moho) and direct S-wave: {}\n'.format(t5SH[0] - tS[0]))
output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
    (t5SH[1] - tS[1]) - (t5SH[0] - tS[0]), \
    (t5SH[2] - tS[2]) - (t5SH[0] - tS[0]), \
    (t5SH[3] - tS[3]) - (t5SH[0] - tS[0])))
output.write('\n')

# Reflected wave on mid-slab discontinuity
# Downgoing wave in upper oceanic crust
for k1 in range(0, 2):
    # Upgoing wave in upper oceanic crust
    for k2 in range(0, 2):
        # Upgoing wave in continental crust
        for k3 in range(0, 2):
            k = k1 * 1 + k2 * 2 + k3 * 4
            output.write('Amplitude ratio of ray {}{}{} / direct S-wave: {}\n'.format(wave[k1], wave[k2], wave[k3], \
                A3PSV[k] / ASV))
            output.write('Time lag between ray {}{}{} and direct S-wave: {}\n'.format(wave[k1], wave[k2], wave[k3], \
                t3PSV[0, k] - tS[0]))
            output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                (t3PSV[1, k] - tS[1]) - (t3PSV[0, k] - tS[0]), \
                (t3PSV[2, k] - tS[2]) - (t3PSV[0, k] - tS[0]), \
                (t3PSV[3, k] - tS[3]) - (t3PSV[0, k] - tS[0])))
            output.write('\n')

# Reflected wave on Moho
# Downgoing wave in upper oceanic crust
for k1 in range(0, 2):
    # Downgoing wave in lower oceanic crust
    for k2 in range(0, 2):
        # Upgoing wave in lower oceanic crust
        for k3 in range(0, 2):
            # Upgoing wave in upper oceanic crust
            for k4 in range(0, 2):
                # Upgoing wave in continental crust
                for k5 in range(0, 2):
                    k = k1 * 1 + k2 * 2 + k3 * 4 + k4 * 8 + k5 * 16
                    output.write('Amplitude ratio of ray {}{}{}{}{} / direct S-wave: {}\n'.format(wave[k1], wave[k2], \
                        wave[k3], wave[k4], wave[k5], A5PSV[k] / ASV))
                    output.write('Time lag between ray {}{}{}{}{} and direct S-wave: {}\n'.format(wave[k1], wave[k2], \
                        wave[k3], wave[k4], wave[k5], t5PSV[0, k] - tS[0]))
                    output.write('Difference in time lag {} s (500m), {} s (1800m), {} s (3300m)\n'.format( \
                        (t5PSV[1, k] - tS[1]) - (t5PSV[0, k] - tS[0]), \
                        (t5PSV[2, k] - tS[2]) - (t5PSV[0, k] - tS[0]), \
                        (t5PSV[3, k] - tS[3]) - (t5PSV[0, k] - tS[0])))
                    output.write('\n')

Close ouput file.

In [13]:
output.close()

We may see on the horizontal autocorrelation plot:
- Direct S-wave and reflected SH-wave (mid-slab): 3.03 s

We may see on the vertical autocorrelation plot:
- Direct P-wave and reflected PPP-wave: 1.34 s

We may see on the cross correlation plot:
- Direct S-wave and direct P-wave: 5.33 s
- Direct P-wave and reflected SH-wave (mid-slab): 8.36 s
- Direct P-wave and reflected SSS wave: 8.36 s
- Direct S-wave and reflected wave PPP: 3.99 s